In [1]:
!pip install psycopg[binary]


In [2]:
!pip show psycopg


Name: psycopg
Version: 3.2.12
Summary: PostgreSQL database adapter for Python
Home-page: https://psycopg.org/psycopg3/
Author: Daniele Varrazzo
Author-email: daniele.varrazzo@gmail.com
License-Expression: LGPL-3.0-only
Location: C:\Users\Administrateur\Chatbot-RAG\venv\Lib\site-packages
Requires: typing-extensions, tzdata
Required-by: 


In [3]:
import psycopg

# Connexion à la base rag_chatbot
conn = psycopg.connect(
    dbname="rag_chatbot",
    user="postgres",
    password="polyvalent00",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Création de la table documents
create_table_query = """
CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    corpus TEXT,
    embedding FLOAT
);
"""
cur.execute(create_table_query)
conn.commit()
print("Table 'documents' créée avec succès.")
cur.close()
conn.close()


Table 'documents' créée avec succès.


In [ ]:
import os

data_dir = "../data/TRANS_TXT"

for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        path = os.path.join(data_dir, filename)
        with open(path, "r", encoding="latin-1") as f:
            text = f.read()
            print(text[:50])  # aperçu des 50 premiers caractères


<01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e
<03> hotesse+client
     h: je vous écoute
     c: c'est pour
<04> client
     c: e c'est pour savoir si la fac pendant l'été e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol
<05> hotesse
     h: e ce serait pour vous vous souhaiteriez
<06> hotesse+client
     h: non
     c: non non c'est pas pour moi
<07> client
     c: 
<01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour ce serait pour savoir comment obtenir e un dossier de candidature pour le D E S S e droit des affaires droit des P M E
<03> hotesse
     h: e oui je vais vous passer le secrétariat
<04> client
     c: d'accord merci
<05> hotesse
     h: [pi]
<06> hotesse+hotesse2
     h: Prenom oui
     h: allo oui
<07> hotesse
     h: un dossier e D E S S droit des P M E
<08> hotesse2
     h: d'accord

<01> hotesse
     h: U B S bonjour
<02> cl

In [7]:
!pip install langchain-community
!pip install langchain-core
!pip install langchain
!pip install langchain-ollama


  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_classic-1.0.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached sqlalchemy-2.0.44-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.13.2-cp312-cp312-win_amd64.whl.metadata (8.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached langsmith-0.4.43-py3-none-any.whl.metadata (14 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.8.0-cp312-cp312-win_amd64.whl.metadata (21 kB)
  Using cached multidict-6.7.0-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached propcache-0.4.1-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached yarl-1.22.0-cp312-cp312-win_amd64.whl.meta

In [8]:
from langchain_community.llms import Ollama

llama_llm = Ollama(model="mistral", temperature=0)

system_message = """
Classify the sentiment of the review presented in the input as 'positive' or 'negative'.
The review will be delimited by triple backticks ``` in the input.
Answer only 'positive' or 'negative'.
Do not explain your answer.
"""

user_message_template = "```{review}```"
user_review = "The look is bad"

few_shot_prompt = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message_template.format(review=user_review)},
]

response = llama_llm.invoke(few_shot_prompt)
print(response)


C:\Users\Administrateur\AppData\Local\Temp\ipykernel_15024\2082741707.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llama_llm = Ollama(model="mistral", temperature=0)


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002874E0BF6B0>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

In [9]:
import psycopg

# Connexion à la base rag_chatbot
conn = psycopg.connect(
    dbname="rag_chatbot",
    user="postgres",
    password="polyvalent00",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Activer l'extension vector (à faire une seule fois)
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
print("Extension vector activée avec succès!")

# Recréer la table avec le type VECTOR
create_table_query = """
DROP TABLE IF EXISTS documents;
CREATE TABLE documents (
    id SERIAL PRIMARY KEY,
    corpus TEXT,
    embedding VECTOR(384)  -- 384 dimensions pour all-miniLM-L6-v2
);
"""
cur.execute(create_table_query)
conn.commit()
print("Table 'documents' recréée avec le type VECTOR.")

cur.close()
conn.close()

Extension vector activée avec succès!
Table 'documents' recréée avec le type VECTOR.


In [12]:
# Télécharger et installer Ollama depuis https://ollama.ai/
# Puis télécharger les modèles
!ollama pull all-miniLM-l6-v2
!ollama pull llama2

'ollama' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'ollama' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
